In [1]:
from Nb_utils import *
from FLpkg import FedUtils


/home/superorange5/.local/lib/python3.7/site-packages/torchvision/transforms/transforms.py:892: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


In [4]:
model_city_8class = load_sourceonlyModel('configs/PFRCNN/PFRCNN_c2b_8cla.yaml', 'output/PFRCNN_c2b_8class/model_final.pth')


-------- pretrained model loaded ---------


In [5]:
model_kitti_1class = load_sourceonlyModel('keep_experiments/c2b_source-only/cfg.yaml', 'keep_experiments/c2b_source-only/model_final.pth')


-------- pretrained model loaded ---------


In [6]:
model_list = [model_kitti_1class,model_city_8class]

In [7]:
wk_ratio=[0.5,0.5]

In [36]:
avg_model = avgWeight_backbone_only(model_list, wk_ratio,"backbone")



cuda:0
cuda:0
backbone.vgg_block1.0.conv1.weight
backbone.vgg_block1.0.conv1.bias
backbone.vgg_block1.0.conv2.weight
backbone.vgg_block1.0.conv2.bias
backbone.vgg_block2.0.conv1.weight
backbone.vgg_block2.0.conv1.bias
backbone.vgg_block2.0.conv2.weight
backbone.vgg_block2.0.conv2.bias
backbone.vgg_block3.0.conv1.weight
backbone.vgg_block3.0.conv1.bias
backbone.vgg_block3.0.conv2.weight
backbone.vgg_block3.0.conv2.bias
backbone.vgg_block3.0.conv3.weight
backbone.vgg_block3.0.conv3.bias
backbone.vgg_block4.0.conv1.weight
backbone.vgg_block4.0.conv1.bias
backbone.vgg_block4.0.conv2.weight
backbone.vgg_block4.0.conv2.bias
backbone.vgg_block4.0.conv3.weight
backbone.vgg_block4.0.conv3.bias
backbone.vgg_block5.0.conv1.weight
backbone.vgg_block5.0.conv1.bias
backbone.vgg_block5.0.conv2.weight
backbone.vgg_block5.0.conv2.bias
backbone.vgg_block5.0.conv3.weight
backbone.vgg_block5.0.conv3.bias


## FedUtils.avgWeight

In [34]:
def avgWeight_backbone_only(model_list,ratio_list,keywords=None):
    parties = len(model_list)
    model_tmp=[None] * parties
    #optims_tmp=[None] * parties

    for idx, my_model in enumerate(model_list):
        print(next(my_model.parameters()).device)
        
        model_tmp[idx] = my_model.state_dict()


    for key in model_tmp[0]:    
        
        if (keywords is None) or (keywords in key):
            print(key)
            model_avg = 0

            for idx, model_tmp_content in enumerate(model_tmp):     # add each model    

                model_avg += ratio_list[idx] * model_tmp_content[key]

            for i in range(len(model_tmp)):  #copy to each model            
                model_tmp[i][key] = model_avg
    for i in range(len(model_list)):    
        model_list[i].load_state_dict(model_tmp[i])
        
    return model_list  #, optims_tmp

In [26]:
avg_model[0].state_dict()['roi_heads.box_predictor.cls_score.weight']

tensor([[-0.0679,  0.0074,  0.0367,  ..., -0.0109,  0.0923, -0.0363],
        [ 0.0606, -0.0137, -0.0236,  ..., -0.0258, -0.0885,  0.0249]],
       device='cuda:0')

In [27]:
avg_model[1].state_dict()['roi_heads.box_predictor.cls_score.weight']

tensor([[-0.0269,  0.0060,  0.0254,  ..., -0.0032, -0.0002, -0.0129],
        [ 0.0084,  0.0230, -0.0476,  ...,  0.0060,  0.0124, -0.0211],
        [ 0.0179,  0.0317, -0.0233,  ..., -0.0057,  0.0105,  0.0063],
        ...,
        [-0.0050, -0.0187,  0.0331,  ...,  0.0146,  0.0160, -0.0013],
        [ 0.0037, -0.0037, -0.0004,  ..., -0.0103,  0.0259, -0.0025],
        [ 0.0211,  0.0007,  0.0490,  ...,  0.0197,  0.0144,  0.0595]],
       device='cuda:0')

## FedMA check

In [49]:

FMA_city = load_sourceonlyModel('configs/PFRCNN/PFRCNN_c2b_8cla.yaml', 'output/FedMA_ck2b_cla8/VOC2007_citytrain_6/model_final.pth')
FMA_kitti = load_sourceonlyModel('configs/PFRCNN/PFRCNN_c2b_8cla.yaml', 'output/FedMA_ck2b_cla8/VOC2007_kitti1_6/model_final.pth')

FMA_final = load_sourceonlyModel('configs/PFRCNN/PFRCNN_c2b_8cla.yaml', 'output/FedMA_ck2b_cla8/FedAvg_final.pth')

-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


In [12]:
def load_SOmodel(cfg_path, model_path):
    print("load source-only pt model")
    Trainer= PTrainer_sourceonly   
    if cfg.FEDSET.DYNAMIC: 
        fedma_model = torch.load(model_path)
        backbone_dim = FedUtils.get_backbone_shape(fedma_model)

        cfg.defrost()
        #cfg.MODEL.WEIGHTS = model_path                    
        cfg.BACKBONE_DIM = backbone_dim        
        cfg.freeze

        model = Trainer.build_model(cfg,cfg.BACKBONE_DIM,False)
    else:
        model = Trainer.build_model(cfg) 
    DetectionCheckpointer(model).resume_or_load(model_path, resume=False)
    return model

* check class number of FedMA output model in client1 (KITTI)
* should be 1 class

In [20]:
cfg = setup_all('configs/FedMA/ck2b_FedMA_8cla.yaml')  
cfg.defrost()
cfg.FEDSET.DYNAMIC=True
cfg.MODEL.ROI_HEADS.NUM_CLASSES =1
cfg.freeze()
FMA_kitti = load_SOmodel(cfg, 'output/FedMA_ck2b_cla8/FedMA_VOC2007_kitti1_6.pth')
#len(FMA_kitti.state_dict()['roi_heads.box_predictor.cls_score.weight'])



#     print(len(model.state_dict()['roi_heads.box_predictor.cls_score.weight']))
print(FMA_kitti.state_dict()['backbone.vgg_block3.0.conv2.bias'].shape)
print(FMA_kitti.state_dict()['backbone.vgg_block3.0.conv2.weight'].shape)

load source-only pt model
-------- pretrained model loaded ---------


Shape of backbone.vgg_block3.0.conv2.bias in checkpoint is torch.Size([259]), while shape of backbone.vgg_block3.0.conv2.bias in model is torch.Size([256]).
backbone.vgg_block3.0.conv2.bias will not be loaded. Please double check and see if this is desired.
Shape of backbone.vgg_block3.0.conv2.weight in checkpoint is torch.Size([259, 256, 3, 3]), while shape of backbone.vgg_block3.0.conv2.weight in model is torch.Size([256, 256, 3, 3]).
backbone.vgg_block3.0.conv2.weight will not be loaded. Please double check and see if this is desired.
Shape of backbone.vgg_block3.0.conv3.weight in checkpoint is torch.Size([256, 259, 3, 3]), while shape of backbone.vgg_block3.0.conv3.weight in model is torch.Size([256, 256, 3, 3]).
backbone.vgg_block3.0.conv3.weight will not be loaded. Please double check and see if this is desired.
Skip loading parameter 'backbone.vgg_block3.0.conv2.bias' to the model due to incompatible shapes: (259,) in the checkpoint but (256,) in the model! You might want to dou

torch.Size([256])
torch.Size([256, 256, 3, 3])


* check the initial model of FedMA (teacher)

In [2]:
model_kitti_1class = load_sourceonlyModel('configs/PFRCNN/PFRCNN_k2b_1cla.yaml', 'output/PFRCNN_k2b_1class/model_final.pth')
len(model_kitti_1class.state_dict()['roi_heads.box_predictor.cls_score.weight'])

-------- pretrained model loaded ---------


2

In [10]:
print(model_kitti_1class.state_dict()['backbone.vgg_block3.0.conv2.bias'].shape)
print(model_kitti_1class.state_dict()['backbone.vgg_block3.0.conv2.weight'].shape)

torch.Size([256])
torch.Size([256, 256, 3, 3])


In [122]:
cfg = setup_all('configs/FedMA/ck2b_FedMA_8cla.yaml') 
teacher_trainer = cfg.MODEL.TEACHER_TRAINER
model_path_list = cfg.MODEL.TEACHER_PATH    

#model_path_list = ['output/PFRCNN_c2b_8class/model_final.pth', 'keep_experiments/k2c_source-only/model_final.pth']
print(model_path_list)

['output/PFRCNN_c2b_8class/model_final.pth', 'output/PFRCNN_k2b_1class/model_final.pth']


In [123]:
from train_net_FedMA import get_trainer
PFRCNN_8class_city_model = get_trainer(teacher_trainer, cfg, model_path_list[0])
PFRCNN_1class_kitti_model = get_trainer(teacher_trainer, cfg, model_path_list[1])

load source-only pt model
-------- pretrained model loaded ---------
load source-only pt model
-------- pretrained model loaded ---------


roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


In [124]:

len(PFRCNN_1class_kitti_model.state_dict()['roi_heads.box_predictor.cls_score.weight'])

9

In [113]:
len(torch.load(model_path_list[1])['model']['modelStudent.roi_heads.box_predictor.cls_score.weight'])

2

In [112]:
len(torch.load(model_path_list[1])['model']['modelTeacher.roi_heads.box_predictor.cls_score.weight'])

2

* teacher & student都是1 class, 8 class是因為讀錯model造成的?

In [94]:
model_list=[ get_trainer(teacher_trainer, cfg, model_teacher_path) for model_teacher_path in model_path_list]


load source-only pt model
-------- pretrained model loaded ---------
load source-only pt model
-------- pretrained model loaded ---------


backbone.vgg_block1.0.conv1.{bias, weight}
backbone.vgg_block1.0.conv2.{bias, weight}
backbone.vgg_block2.0.conv1.{bias, weight}
backbone.vgg_block2.0.conv2.{bias, weight}
backbone.vgg_block3.0.conv1.{bias, weight}
backbone.vgg_block3.0.conv2.{bias, weight}
backbone.vgg_block3.0.conv3.{bias, weight}
backbone.vgg_block4.0.conv1.{bias, weight}
backbone.vgg_block4.0.conv2.{bias, weight}
backbone.vgg_block4.0.conv3.{bias, weight}
backbone.vgg_block5.0.conv1.{bias, weight}
backbone.vgg_block5.0.conv2.{bias, weight}
backbone.vgg_block5.0.conv3.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
  modelTeacher.backbone.vgg_block1.0.conv1.{bias, weight}
  modelTeacher.backbone.vgg_block1.0.conv2.

In [81]:
model_kitti_1class = load_sourceonlyModel('keep_experiments/c2b_source-only/cfg.yaml', 'keep_experiments/c2b_source-only/model_final.pth')


load source-only pt model


In [82]:
len(FMA_kitti.state_dict()['roi_heads.box_predictor.cls_score.weight'])

9

In [47]:
FMA_final.state_dict()

OrderedDict([('backbone.vgg_block1.0.conv1.weight',
              tensor([[[[ 4.2947e-01,  3.7347e-01, -6.1360e-02],
                        [ 2.7477e-01,  3.8681e-02, -3.6722e-01],
                        [-5.7468e-02, -2.6225e-01, -3.5010e-01]],
              
                       [[ 5.5038e-01,  4.4007e-01, -8.1387e-02],
                        [ 3.4574e-01,  4.0632e-02, -4.5350e-01],
                        [-5.8635e-02, -3.3067e-01, -4.8503e-01]],
              
                       [[ 4.8002e-01,  4.0855e-01, -6.5146e-02],
                        [ 3.1048e-01,  5.0202e-02, -4.0338e-01],
                        [-5.0872e-02, -2.8523e-01, -4.1852e-01]]],
              
              
                      [[[ 1.1727e-01,  1.6206e-01,  1.3569e-01],
                        [ 1.4835e-01,  2.0230e-01,  1.6169e-01],
                        [ 1.2934e-01,  1.7157e-01,  1.3871e-01]],
              
                       [[ 2.0877e-02,  4.7341e-02,  4.1854e-02],
                       